In [ ]:
import subprocess
import os
import re
from bs4 import BeautifulSoup
import pandas as pd

# --- 1. 配置 ---
SOURCE_DOC_FILE = "F:\\福卡\\福卡知识库测试文件\\4资本金融\\4-5科技金融\\20180425数字货币、加密货币、区块链甚嚣尘上————投机资本真的没方向.doc" 
# 自动将输出Excel放在源文档同一目录
OUTPUT_EXCEL_FILE = os.path.join(os.path.dirname(os.path.abspath(SOURCE_DOC_FILE)), 
                                os.path.basename(SOURCE_DOC_FILE).rsplit('.', 1)[0] + '.xlsx')
# 切割配置
MAX_CHUNK_LEN = 500  # 合并的目标上限
MIN_CHUNK_LEN = 100  # 合并的最小阈值
HARD_MAX_LEN = 1000 # 单个段落的硬性拆分上限


# --- 2. 文档加载 ---
def find_libreoffice():
    possible_paths = [
        r"C:\\Program Files\\LibreOffice\\program\\soffice.exe",
        r"C:\\Program Files (x86)\\LibreOffice\\program\\soffice.exe",
        r"D:\\LibreOffice\\program\\soffice.exe",
        "soffice"
    ]
    for path in possible_paths:
        try:
            if path != "soffice" and not os.path.exists(path): continue
            result = subprocess.run([path, '--version'], capture_output=True, text=True, timeout=10)
            if result.returncode == 0: return path
        except: continue
    return None

def load_doc_as_text(doc_path):
    print("正在加载文档...")
    libreoffice_path = find_libreoffice()
    if not libreoffice_path:
        print("错误：未找到 LibreOffice (soffice.exe)，请检查路径。")
        return None
    
    doc_path = os.path.abspath(doc_path)
    output_dir = os.path.dirname(doc_path)
    html_filename = os.path.basename(doc_path).rsplit('.', 1)[0] + '.html'
    html_path = os.path.join(output_dir, html_filename)
    
    if os.path.exists(html_path): 
        try: os.remove(html_path)
        except: pass
        
    cmd = [libreoffice_path, '--headless', '--convert-to', 'html', '--outdir', output_dir, doc_path]
    subprocess.run(cmd, capture_output=True, text=True)
    
    if not os.path.exists(html_path):
        print(f"错误：LibreOffice 转换失败，未在 {output_dir} 找到 {html_filename}")
        return None
    
    content = ""
    for enc in ['utf-8', 'gb18030', 'gbk']:
        try:
            with open(html_path, 'r', encoding=enc) as f:
                content = f.read()
                break
        except: continue
            
    if not content:
        print("错误：读取转换后的 HTML 文件失败。")
        return None

    soup = BeautifulSoup(content, 'html.parser')
    
    # 1. 移除页眉和页脚的 div
    print("   -> 正在移除页眉 (header) 和页脚 (footer)...")
    header = soup.find('div', title='header')
    if header:
        header.decompose() 

    footer = soup.find('div', title='footer')
    if footer:
        footer.decompose()
    
    # 2. 获取所有非空段落
    # 获取所有可能的文本标签，不仅仅是 p
    # 包含段落(p), 各种标题(h1-h6), 分块(div), 列表项(li)
    target_tags = ['p', 'div', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'li', 'blockquote']
    found_tags = soup.find_all(target_tags)
    
    cleaned_paragraphs = []
    for tag in found_tags:
        # 1. 获取文本
        p_text = tag.get_text()
        
        # 2. 移除标签内部的换行符
        # LibreOffice 会在 HTML 标签内部插入 \n 进行视觉换行，
        # 这对于中文文档会导致句子被错误切断。必须将其替换为空字符串。
        p_text = p_text.replace('\n', '').replace('\r', '')
        
        # 3. 去除首尾空白
        p_text = p_text.strip()
        
        if p_text:
            cleaned_paragraphs.append(p_text)

    # 3. 找到正文开始的标记 "〖特别报告〗"，删除之前的内容
    start_index = -1
    for i, p_text in enumerate(cleaned_paragraphs):
        if "〖特别报告〗" in p_text:
            start_index = i
            break
    
    final_paragraphs = []
    if start_index != -1:
        # 找到了标记
        print(f"   -> 找到正文起始标记 '〖特别报告〗' (位于段落 {start_index})。")
        
        # 3a. 获取 *第一个* 正文段落
        first_para_text = cleaned_paragraphs[start_index]
        
        # 3b. 【新】使用 .split() 来切分，这比 .find() 更健壮
        # 这将创建列表: ["福卡分析...福卡理念：...", " 世界改造中国..."]
        parts = first_para_text.split("〖特别报告〗", 1)
        
        if len(parts) == 2:
            # 成功切分. parts[0] 是垃圾页眉, parts[1] 是正文.
            # 我们重新组合，保留标记和正文。
            cleaned_first_para = "〖特别报告〗" + parts[1]
            final_paragraphs.append(cleaned_first_para.strip()) # 添加清理后的第一段
            print(f"   -> 已成功清理正文标题行。")
        else:
            # 切分失败（极不可能，但作为保险）
            print("   -> 警告: 'split' 失败，按原样保留段落。")
            final_paragraphs.append(first_para_text)
            
        # 3c. 添加所有剩余的段落
        final_paragraphs.extend(cleaned_paragraphs[start_index + 1:])
        print(f"   -> 已处理 {len(final_paragraphs)} 个正文段落。")

    else:
        # 没找到标记
        print("   -> 警告：未找到 '〖特别报告〗' 标记，将处理所有段落。")
        final_paragraphs = cleaned_paragraphs
    
    # 4. 用单一的、可靠的 \n 将所有“干净”的段落连接起来
    text = "\n".join(final_paragraphs)
    
    try: 
        os.remove(html_path) 
        print("   -> 中间 HTML 文件已清理。")
    except: 
        pass
    
    return text


# --- 3. 语义切割逻辑 ---
def split_long_paragraph(text, hard_max):
    print(f"   -> 检测到长段落 (长度 {len(text)})，尝试按中间句号 '。' 切分...")
    mid_point = len(text) // 2
    
    pos_before = text.rfind('。', 0, mid_point)
    pos_after = text.find('。', mid_point)
    
    split_pos = -1
    
    if pos_before != -1 and pos_after != -1:
        if (mid_point - pos_before) < (pos_after - mid_point):
            split_pos = pos_before
        else:
            split_pos = pos_after
    elif pos_before != -1:
        split_pos = pos_before
    elif pos_after != -1:
        split_pos = pos_after
    else:
        print(f"   -> 警告: 段落长度 {len(text)} > {hard_max}，但未找到 '。' 无法切分。")
        return [text] 
    
    part1 = text[:split_pos + 1].strip()
    part2 = text[split_pos + 1:].strip()
    
    if not part1 or not part2:
         print(f"   -> 警告: 按句号切分失败 (产生空块)，返回原长段落。")
         return [text]
         
    print(f"   -> S 成功切分为两块: (长度 {len(part1)}) 和 (长度 {len(part2)})")
    return [part1, part2]


def split_text_smart(text, max_len=MAX_CHUNK_LEN, min_len=MIN_CHUNK_LEN):
    print(f"正在按段落切割 (合并目标: {max_len}, 最小: {min_len}, 硬上限: {HARD_MAX_LEN})...")
    
    # 1. 移除"相关链接"
    text = re.split(r"〖相关链接：信息〗|〖相关链接：报告〗|〖参考信息〗|〖参考报告〗", text, 1)[0]
    
    # 2. 基础清理
    text = re.sub(r"\"", "", text)
    text = re.sub(r"[\x00-\x08\x0B\x0C\x0E-\x1f\x7f-\x9f]", "", text) # 保留 \n \r \t
    text = re.sub(r"[\r\n\t ]+\n", "\n", text)
    text = re.sub(r"\n\s*\n", "\n", text).strip()
    
    paragraphs = text.split('\n')
    paragraphs = [p.strip() for p in paragraphs if p.strip()]

    # 3. 长度控制
    initial_chunks = []
    current_chunk = ""
    
    for para in paragraphs:
        # 3a. 检查 > 1000
        if len(para) > HARD_MAX_LEN:
            sub_paragraphs = split_long_paragraph(para, HARD_MAX_LEN)
        else:
            sub_paragraphs = [para]

        # 3b. 迭代处理(可能被切分后)的段落，并应用 < 500 合并逻辑
        for sub_para in sub_paragraphs:
            if not sub_para.strip(): continue 
            
            if not current_chunk:
                current_chunk = sub_para
            elif len(current_chunk) + len(sub_para) + 1 <= max_len: 
                current_chunk += "\n" + sub_para
            else:
                initial_chunks.append(current_chunk)
                current_chunk = sub_para
    
    if current_chunk:
        initial_chunks.append(current_chunk)
        
    print(f"初步切割得到 {len(initial_chunks)} 个块，正在进行小块合并...")

    # 4. 后处理：合并 < 100 的块
    final_chunks = []
    i = 0
    while i < len(initial_chunks):
        current_chunk = initial_chunks[i]
        
        if i == 0 and len(current_chunk) < min_len:
             print(f"   -> 提示：第一个块 (Chunk 0) 长度 {len(current_chunk)} < {min_len}。")
        
        if len(current_chunk) < min_len and (i < len(initial_chunks) - 1):
            next_chunk = initial_chunks[i+1]
            merged_chunk = current_chunk + "\n" + next_chunk
            
            if len(merged_chunk) <= max_len:
                print(f"   -> (Chunk {i}) 长度 {len(current_chunk)} 太小，已与下一块合并。")
                final_chunks.append(merged_chunk)
                i += 2 
            else:
                print(f"   -> (Chunk {i}) 长度 {len(current_chunk)} 太小，但无法与下一块合并 (会超长)。")
                final_chunks.append(current_chunk)
                i += 1
        else:
            final_chunks.append(current_chunk)
            i += 1

    print(f"清理完成，最终得到 {len(final_chunks)} 个文本块。")
    return final_chunks


# --- 4. 主程序 ---
def main():
    # 确保文件存在
    if not os.path.exists(SOURCE_DOC_FILE):
        print(f"❌ 错误：源文件未找到！")
        print(f"请检查 'SOURCE_DOC_FILE' 变量是否指向正确的文件：")
        print(f"{os.path.abspath(SOURCE_DOC_FILE)}")
        return
        
    full_text = load_doc_as_text(SOURCE_DOC_FILE)
    if not full_text: 
        print("❌ 错误：未能加载文档。")
        return

    # 1. 执行切割
    chunks = split_text_smart(full_text)
    
    if not chunks:
        print("❌ 错误：切割后未产生任何文本块。")
        return

    # 2. 预览检查
    print("\n" + "="*30)
    print("      最终切分效果预览      ")
    print("="*30)
    
    for i, chunk in enumerate(chunks):
        if i < 5: 
            print(f"\n>>> Chunk {i} (长度: {len(chunk)})")
            preview = chunk.replace("\n", " ")[:80]
            print(f"内容: {preview}...")
            
    if len(chunks) > 5:
        print(f"\n... (及其他 {len(chunks) - 5} 个块)")
    
    
    # 3. 转换为 DataFrame
    print(f"\n正在将 {len(chunks)} 个块转换为 DataFrame...")
    
    df = pd.DataFrame(chunks, columns=['text_content'])
    df['length'] = df['text_content'].str.len()
    df['chunk_id'] = range(1, len(df) + 1)
    
    # 调整列顺序
    df = df[['chunk_id', 'text_content', 'length']]
    
    # 4. 导出到 Excel
    try:
        df.to_excel(OUTPUT_EXCEL_FILE, index=False, engine='openpyxl')
        print(f"\n✅ 任务完成！已成功导出 {len(df)} 条数据到：")
        print(f"{os.path.abspath(OUTPUT_EXCEL_FILE)}")
    except Exception as e:
        print(f"\n❌ 导出 Excel 失败：{e}")
        print("提示：如果文件已打开，请关闭它再重试。")
    
# --- 运行 ---
if __name__ == "__main__":
    main()

正在加载文档...
   -> 正在移除页眉 (header) 和页脚 (footer)...
   -> 找到正文起始标记 '〖特别报告〗' (位于段落 0)。
   -> 已成功清理正文标题行。
   -> 已处理 23 个正文段落。
   -> 中间 HTML 文件已清理。
正在按段落切割 (合并目标: 500, 最小: 100, 硬上限: 1000)...
初步切割得到 5 个块，正在进行小块合并...
   -> 提示：第一个块 (Chunk 0) 长度 90 < 100。
   -> (Chunk 0) 长度 90 太小，但无法与下一块合并 (会超长)。
清理完成，最终得到 5 个文本块。

      最终切分效果预览      

>>> Chunk 0 (长度: 90)
内容: 〖特别报告〗数字货币、加密货币、区块链甚嚣尘上 ——投机资本真的没方向 摘要：数字货币、加密货币、区块链等概念诞生于码农圈，成长于币圈，深入影响了互联网圈，又在...

>>> Chunk 1 (长度: 553)
内容: 币圈火了、疯了、乱了。2017年内比特币、莱特币、以太币分别暴涨14倍、76倍、80倍，名不见经传的瑞波币连涨360倍，一时风头无两；但到了2018年，行情斗转...

>>> Chunk 2 (长度: 786)
内容: 一时间，加密货币、Token、区块链概念惹得满城风雨，屡被混为一谈，其中关系剪不断、理还乱，为贪婪的庄家和投机的资本玩家提供了疯狂敛收智商税的好机会。与此同时，...

>>> Chunk 3 (长度: 911)
内容: （3）数字货币一般是指以数字形式存在的法定货币，由国家做信用背书，有价值锚定，具备信用创造功能，对经济产生实质作用。在加密货币颠覆货币体系的言论甚嚣尘上之时，为...

>>> Chunk 4 (长度: 718)
内容: 尽管加密货币的用户群体只有2000万，交易量只有外汇交易量的十万分之一，在峰值时的总市值尚且不到全球GDP的1%，但正如IMF总裁拉加德所描绘，“我们将见证异常...

正在将 5 个块转换为 DataFrame...

✅ 任务完成！已成功导出 5 条数据到：
F:\福卡\福卡知识库测试文件\4资本金融\4-5科技金融\20180425数字货币、加密货币、区块链甚嚣尘上————投机资本真的没方向.xl